# ChAII - Predict

In [1]:
import pandas as pd
import collections
import numpy as np
!pip uninstall fsspec -qq -y
!pip install --no-index --find-links ../input/hf-datasets/wheels datasets -qq
from datasets import Dataset

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-cloud 0.1.13 requires tensorflow<3.0,>=1.15.0, which is not installed.
dask-cudf 21.6.1+2.g101fc0fda4 requires cupy-cuda112, which is not installed.
cudf 21.6.1+2.g101fc0fda4 requires cupy-cuda110, which is not installed.
s3fs 2021.6.1 requires fsspec==2021.06.1, but you have fsspec 2021.6.0 which is incompatible.
pytorch-lightning 1.3.8 requires fsspec[http]!=2021.06.0,>=2021.05.0, but you have fsspec 2021.6.0 which is incompatible.
dask-cudf 21.6.1+2.g101fc0fda4 requires dask<=2021.5.1,>=2021.4.0, but you have dask 2021.6.2 which is incompatible.
dask-cudf 21.6.1+2.g101fc0fda4 requires distributed<=2021.5.1,>=2.22.0, but you have distributed 2021.6.2 which is incompatible.


In [2]:
sub = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/sample_submission.csv')
sub

,id,PredictionString
0,22bff3dec,NaN
1,282758170,NaN
2,d60987e0e,NaN
3,f99c770dc,NaN
4,40dec1964,NaN


### What about test? 

In test, we have just 5 examples, and similar columns like in train except that we don't have *answer_text* and *answer_start*. It seems that *answer_text* is what we're trying to predict. 

In [3]:
test = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/test.csv')
test.head()

,id,context,question,language
0,22bff3dec,"ज्वाला गुट्टा (जन्म: 7 सितंबर 1983; वर्धा, महा...",ज्वाला गुट्टा की माँ का नाम क्या है,hindi
1,282758170,गूगल मानचित्र (Google Maps) (पूर्व में गूगल लो...,गूगल मैप्स कब लॉन्च किया गया था?,hindi
2,d60987e0e,गुस्ताव रॉबर्ट किरचॉफ़ (१२ मार्च १८२४ - १७ अक्...,गुस्ताव किरचॉफ का जन्म कब हुआ था?,hindi
3,f99c770dc,அலுமினியம் (ஆங்கிலம்: அலுமினியம்; வட அமெரிக்க ...,அலுமினியத்தின் அணு எண் என்ன?,tamil
4,40dec1964,"கூட்டுறவு இயக்க வரலாறு, இங்கிலாந்து நாட்டில் ...",இந்தியாவில் பசுமை புரட்சியின் தந்தை என்று கருத...,tamil


In [4]:
test_hindi = test.loc[test['language'] == 'hindi']
test_tamil = test.loc[test['language'] == 'tamil']
test_hindi

,id,context,question,language
0,22bff3dec,"ज्वाला गुट्टा (जन्म: 7 सितंबर 1983; वर्धा, महा...",ज्वाला गुट्टा की माँ का नाम क्या है,hindi
1,282758170,गूगल मानचित्र (Google Maps) (पूर्व में गूगल लो...,गूगल मैप्स कब लॉन्च किया गया था?,hindi
2,d60987e0e,गुस्ताव रॉबर्ट किरचॉफ़ (१२ मार्च १८२४ - १७ अक्...,गुस्ताव किरचॉफ का जन्म कब हुआ था?,hindi


# Baseline

Based on: https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb

In [5]:
import transformers
from transformers import AutoTokenizer

model_checkpoint = '../input/xlm-roberta-squad2/deepset/xlm-roberta-large-squad2'
batch_size = 4

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
pad_on_right = tokenizer.padding_side == "right"

max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed

In [6]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator

model_hindi = AutoModelForQuestionAnswering.from_pretrained('../input/chaii-eda-baseline/chaii-bert-trained-hindi/')
model_tamil = AutoModelForQuestionAnswering.from_pretrained('../input/chaii-eda-baseline/chaii-bert-trained-tamil/')
data_collator = default_data_collator

In [7]:
# import wandb
# wandb.login()
# %env WANDB_PROJECT=chaii
%env WANDB_DISABLED=True

env: WANDB_DISABLED=True


In [8]:
args = TrainingArguments(
    f"chaii-qa",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    warmup_ratio=0.1,
    gradient_accumulation_steps=8,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
#     report_to="wandb"
)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [9]:
trainer_hindi = Trainer(
    model_hindi,
    args,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer_tamil = Trainer(
    model_tamil,
    args,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [10]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [11]:
max_answer_length = 30

In [12]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        predictions[example["id"]] = best_answer["text"]

    return predictions

## Test predict and submit

In [13]:
test_dataset_hindi = Dataset.from_pandas(test_hindi)
test_dataset_tamil = Dataset.from_pandas(test_tamil)

In [14]:
test_features_hindi = test_dataset_hindi.map(
    prepare_validation_features,
    batched=True,
    remove_columns=test_dataset_hindi.column_names
)

test_features_tamil = test_dataset_tamil.map(
    prepare_validation_features,
    batched=True,
    remove_columns=test_dataset_tamil.column_names
)

In [15]:
test_feats_small_hindi = test_features_hindi.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])
test_feats_small_tamil = test_features_tamil.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])

test_feats_small_hindi

Dataset({
    features: ['attention_mask', 'input_ids'],
    num_rows: 51
})

In [16]:
test_predictions_hindi = trainer_hindi.predict(test_feats_small_hindi)
test_predictions_tamil = trainer_tamil.predict(test_feats_small_tamil)

In [17]:
test_features_hindi.set_format(type=test_features_hindi.format["type"], columns=list(test_features_hindi.features.keys()))
test_features_tamil.set_format(type=test_features_tamil.format["type"], columns=list(test_features_tamil.features.keys()))

In [18]:
final_test_predictions_hindi = postprocess_qa_predictions(test_dataset_hindi, test_features_hindi, test_predictions_hindi.predictions)
final_test_predictions_tamil = postprocess_qa_predictions(test_dataset_tamil, test_features_tamil, test_predictions_tamil.predictions)
final_test_predictions_hindi

Post-processing 3 example predictions split into 51 features.



Post-processing 2 example predictions split into 16 features.


OrderedDict([('22bff3dec', ' येलन'),
             ('282758170', ' 20 अप्रैल 2010'),
             ('d60987e0e', '१२ मार्च १८२४')])

In [19]:
final_test_predictions_hindi = list(final_test_predictions_hindi.items())
final_test_predictions_tamil = list(final_test_predictions_tamil.items())

final_test_predictions = final_test_predictions_hindi + final_test_predictions_tamil
final_test_predictions

[('22bff3dec', ' येलन'),
 ('282758170', ' 20 अप्रैल 2010'),
 ('d60987e0e', '१२ मार्च १८२४'),
 ('f99c770dc', ' 13'),
 ('40dec1964', 'சுவாமிநாதன் மற்றும் வர்கீஸ் குரியன்')]

In [20]:
submit = pd.DataFrame(final_test_predictions, columns = ['id', 'PredictionString'])
submit

,id,PredictionString
0,22bff3dec,येलन
1,282758170,20 अप्रैल 2010
2,d60987e0e,१२ मार्च १८२४
3,f99c770dc,13
4,40dec1964,சுவாமிநாதன் மற்றும் வர்கீஸ் குரியன்


In [21]:
submit.to_csv('submission.csv', index=False)